In [16]:
import numpy as np
import pandas as pd
import re
import os
from collections import Counter
from typing import Set

# 1단계 : 데이터 로딩 및 기본 전처리

### 데이터 로딩

In [17]:
# 소셜 데이터 전체 불러오기(234036)
df = pd.read_csv('../data/raw/통합_소셜_데이터(원본 종합).csv', encoding='utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234036 entries, 0 to 234035
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        234036 non-null  object
 1   category  234036 non-null  object
 2   date      234036 non-null  int64 
 3   title     227043 non-null  object
 4   content   231420 non-null  object
 5   url       234036 non-null  object
 6   source    234036 non-null  object
dtypes: int64(1), object(6)
memory usage: 12.5+ MB


### 결측치 처리

In [18]:
# 제목과 content에서 공백만 있는 경우 NaN으로 변환

    # 변환 전 결측치
before_missing = df.isnull().sum()

    # 제목과 content에서 공백만 있는 경우 NaN으로 변환
df['title'] = df['title'].replace(r'^\s*$', np.nan, regex=True)
df['content'] = df['content'].replace(r'^\s*$', np.nan, regex=True)

    # 변환 후 결측치 개수 기록 
after_missing = df.isnull().sum()

    # 차이 계산
diff_missing = after_missing - before_missing

    # 보기 좋게 합치기
missing_report = pd.DataFrame({
    "변환 전": before_missing,
    "변환 후": after_missing,
    "증가분": diff_missing
})
print(missing_report)

          변환 전  변환 후  증가분
id           0     0    0
category     0     0    0
date         0     0    0
title     6993  6997    4
content   2616  2669   53
url          0     0    0
source       0     0    0


In [19]:
# title,content 비어있을 경우 상호 채우기

    # title이 비어있으면 content 값으로 채움
title_fill_count = df['title'].isna().sum()
df['title'] = df['title'].fillna(df['content'])

    # content가 비어있으면 title 값으로 채움
content_fill_count = df['content'].isna().sum()
df['content'] = df['content'].fillna(df['title'])

print(f"title이 비어있어서 content로 채운 개수: {title_fill_count}")
print(f"content가 비어있어서 title로 채운 개수: {content_fill_count}")

title이 비어있어서 content로 채운 개수: 6997
content가 비어있어서 title로 채운 개수: 2669


### 텍스트 정제 

In [20]:
# 텍스트 정제
def clean_text(text):
    """
    content나 title 안에서 url, 특수문자, 이모티콘을 제거합니다.
    """
    if pd.isnull(text):
        return ""

    # 1. URL 제거
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', str(text))
    
    # 2. 이모티콘 제거
    emoji_pattern = re.compile("["  
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    # 3. 특수문자 제거 (한글, 영어, 숫자, 공백 제외)
    text = re.sub(r'[^\w\s가-힣]', '', text)
    
    # 4. 여러 개의 공백을 하나로 축소
    text = re.sub(r'\s+', ' ', text).strip()

    # 5. 반복되는 자음, 모음 처리 (예: ㅋㅋㅋㅋ -> ㅋㅋ)
    text = re.sub(r'([ㄱ-ㅎㅏ-ㅣ])\1{2,}', r'\1\1', text)

    # 6. 영문 소문자 변환
    text = text.lower()

    return text

# -------------------------
# df에 정제 결과 반영
# -------------------------
# title과 content 둘 다 정제
df['title'] = df['title'].astype(str).apply(clean_text)
df['content'] = df['content'].astype(str).apply(clean_text)

print(df.head())

                                     id        category            date  \
0  009992f3-67ce-4c9d-87f8-8b70add1dc5b  개인정보보호법,정보통신망법  20250813112502   
1  7321a306-2153-41a8-8426-82ba3b2ac694  개인정보보호법,정보통신망법  20250813105737   
2  29202426-c3bc-4e81-8146-b603869b0801  개인정보보호법,정보통신망법  20250813003504   
3  b554082d-3d28-47e9-8b6a-5335e228ef0c  개인정보보호법,정보통신망법  20250812211635   
4  aff109b1-f50d-4256-bd21-46ca16eaf097  개인정보보호법,정보통신망법  20250812174510   

                                               title  \
0  우왁굳 왁물원 네이버 카페 api 무단 사용개인정보 수집 논란법 위반 소지 개인정보...   
1  형사전문변호사가 말하는 명예훼손죄 성립요건 명예훼손죄는 형법 및 정보통신망법에 따라...   
2  개인정보보호법 제 17조에 따르면 제 3자 즉 공연 주최 측에게 신분증 같은 걸 제...   
3  빅데이터 및 인공지능 시대 정보주체 권리 보호 위한개인정보보호법 개정안발의 기자회견...   
4  유튜브 등의 허위조작정보혐오폭력 선동 콘텐츠는 언론중재법방송법으로는 적용범위가 협소...   

                                             content  \
0  우왁굳 왁물원 네이버 카페 api 무단 사용개인정보 수집 논란법 위반 소지 개인정보...   
1  형사전문변호사가 말하는 명예훼손죄 성립요건 명예훼손죄는 형법 및 정보통신망법에 따라...   
2  개인정보보호법 제 17조에 따르면 제 3자 즉 공연 주최 측에게 신분증 같

### 템플릿 정의 및 제거

In [ ]:
# 템플릿 제거 함수 정의

    # 1. 블랙리스트 생성을 위한 텍스트 정규화 함수
def normalize_text_for_blacklist(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', text)
    text = re.sub(r'[^\w\s가-힣]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip().lower()
    return text

    # 2. df['content']를 직접 받아 블랙리스트를 생성하고, Set으로 바로 반환하는 함수
def create_blacklist_from_content(content_series: pd.Series, min_freq: int, min_len: int) -> Set[str]:
    """
    DataFrame의 'content' Series를 직접 받아 템플릿 블랙리스트(Set)를 생성합니다.
    """
    print("--- df['content']를 기반으로 블랙리스트 생성을 시작합니다. ---")
    
    # 문장 분리
    sentences = content_series.astype(str).str.replace(r'_x000d_', '\n').str.split(r'[.\n!?|;:]|ex\s').explode()
    # 문장 정규화
    normalized_sentences = sentences.apply(normalize_text_for_blacklist)
    # 길이 필터링
    filtered_sentences = normalized_sentences[normalized_sentences.str.len() >= min_len]
    # 빈도수 계산
    sentence_counts = Counter(filtered_sentences)
    # 블랙리스트 생성
    blacklist_set = {sentence for sentence, count in sentence_counts.items() if count >= min_freq}
    
    print(f"블랙리스트 생성 완료: {len(blacklist_set)}개의 템플릿 문장을 찾았습니다.")
    return blacklist_set

    # 3. 생성된 블랙리스트를 적용하여 템플릿 문장을 제거하는 함수
def remove_templates(original_text: str, blacklist: Set[str]) -> str:
    """
    주어진 텍스트에서 블랙리스트에 포함된 문장을 제거합니다.
    """
    if not isinstance(original_text, str):
        return ""
    
    raw_sentences = re.split(r'[.\n!?|;:]|ex\s', original_text)
    clean_sentences = []
    for s in raw_sentences:
        normalized_s = normalize_text_for_blacklist(s)
        if normalized_s and normalized_s not in blacklist:
            clean_sentences.append(s.strip())
            
    return " ".join(clean_sentences)



# 템플릿 제거 실행

# --- 설정값 ---
MIN_FREQUENCY = 50  # 템플릿 문장 최소 빈도
MIN_LENGTH = 15     # 템플릿 문장 최소 길이

# 1. 위에서 전처리된 df['content']를 사용하여 바로 블랙리스트 생성
blacklist_set = create_blacklist_from_content(
    content_series=df['content'],
    min_freq=MIN_FREQUENCY,
    min_len=MIN_LENGTH
)

# 2. [추가] 생성된 블랙리스트를 txt 파일로 저장

output_dir = "../data/processed"

blacklist_filename = os.path.join(output_dir, "blacklistTemplate.txt")
try:
    with open(blacklist_filename, 'w', encoding='utf-8') as f:
        # set은 순서가 없으므로, 정렬하여 저장하면 일관된 결과를 얻을 수 있습니다.
        for sentence in sorted(list(blacklist_set)):
            f.write(sentence + '\n')
    print(f"\n✅ 블랙리스트를 '{blacklist_filename}' 파일로 성공적으로 저장했습니다.")
except Exception as e:
    print(f"\n❗️ 블랙리스트 파일 저장 중 오류가 발생했습니다: {e}")


# 3. 생성된 블랙리스트를 df['title'],df['content']에 바로 적용
print("\n--- 생성된 블랙리스트를 적용하여 템플릿 문장 제거를 시작합니다. ---")
df['title'] = df['title'].astype(str).apply(lambda x: remove_templates(x, blacklist_set))
df['content'] = df['content'].astype(str).apply(lambda x: remove_templates(x, blacklist_set))

print("템플릿 문장 제거 완료.")


# 결과 확인
print("\n--- 템플릿 제거 후 content 열 (상위 5개) ---")
print(df[['content']].head())

# 4. [추가] 생성된 블랙리스트의 상위 5개 샘플을 출력
print("\n--- 생성된 블랙리스트 (상위 5개 샘플) ---")
if blacklist_set:
    # Set은 순서가 없으므로 list로 변환하여 일부를 확인합니다.
    blacklist_sample = list(blacklist_set)[:5]
    for i, sentence in enumerate(blacklist_sample):
        print(f"{i+1}: {sentence}")
else:
    print("생성된 블랙리스트가 없습니다.")

--- df['content']를 기반으로 블랙리스트 생성을 시작합니다. ---
블랙리스트 생성 완료: 62개의 템플릿 문장을 찾았습니다.

✅ 블랙리스트를 '../data/processed\blacklistTempelte.txt' 파일로 성공적으로 저장했습니다.

--- 생성된 블랙리스트를 적용하여 템플릿 문장 제거를 시작합니다. ---
템플릿 문장 제거 완료.

--- 템플릿 제거 후 content 열 (상위 5개) ---
                                             content
0  우왁굳 왁물원 네이버 카페 api 무단 사용개인정보 수집 논란법 위반 소지 개인정보...
1  형사전문변호사가 말하는 명예훼손죄 성립요건 명예훼손죄는 형법 및 정보통신망법에 따라...
2  개인정보보호법 제 17조에 따르면 제 3자 즉 공연 주최 측에게 신분증 같은 걸 제...
3  빅데이터 및 인공지능 시대 정보주체 권리 보호 위한개인정보보호법 개정안발의 기자회견...
4  유튜브 등의 허위조작정보혐오폭력 선동 콘텐츠는 언론중재법방송법으로는 적용범위가 협소...

--- 생성된 블랙리스트 (상위 5개 샘플) ---
1: 디지털성범죄 앞으로 저는 어떻게 될까요
2: 출석 법무법인 에스 유튜브 법률상담은 카카오톡 채팅상담 클릭 강제추행 카메라촬영죄 아청법 아청물소지 n번방 성매매 지하철성추행 공중밀집장소추행 아동청소년강제추행 아청성매수 아동복지법 통매음 음란물유포 토렌트 준강간 압수수색 등
3: 보이스피싱 때문에 경찰서에서 전화왔습니다
4: 안녕하세요 요즘 제일 핫한 중대재해처벌법제조업 방문영업하실 분 모집합니다 단순 방문영업 입니다 중대재해처벌법인 안전보건경영시스템 구축을 갖추면 산재보험료를 20 매년 감면해줍니다 예를 들어 10인 사업장이 1년에 1000만원의 산재보험료를 낸다면 20인 200만원을 매년 감면해 줍니다 자동차보험의 블랙박스를 달면 보험료 7 감면해주는 것과 같은 이치입니다 수도권 및 충청도 방문영업 전화상담 관심 있으신분

# 2단계: 데이터 제거 (중복 및 불필요 데이터) & 광고성 게시물 필터링

### 불필요 데이터 삭제

In [ ]:
# 템플릿 제거 후 content가 NaN, 공백, 또는 'nan' 문자열인 경우 제거
df = df[
    ~(df['content'].isna()) &                                       # 1. NaN 값 제거
    (df['content'].str.strip() != "") &                             # 2. 공백만 있는 경우 제거
    (df['content'].str.strip().str.lower() != 'nan')                # 3. 'nan' 문자열(대소문자 무관) 제거
]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 228803 entries, 0 to 234035
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        228803 non-null  object
 1   category  228803 non-null  object
 2   date      228803 non-null  int64 
 3   title     228803 non-null  object
 4   content   228803 non-null  object
 5   url       228803 non-null  object
 6   source    228803 non-null  object
dtypes: int64(1), object(6)
memory usage: 14.0+ MB


### 완전 빈 데이터 제거

In [23]:
# title과 content가 완전히 일치하는 데이터 중에서, date가 가장 빠른 행만 남기고 나머지 제거
df = df.sort_values('date').drop_duplicates(subset=['title', 'content'], keep='first')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 211261 entries, 163672 to 164761
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        211261 non-null  object
 1   category  211261 non-null  object
 2   date      211261 non-null  int64 
 3   title     211261 non-null  object
 4   content   211261 non-null  object
 5   url       211261 non-null  object
 6   source    211261 non-null  object
dtypes: int64(1), object(6)
memory usage: 12.9+ MB


### 광고성 게시물 필터링

In [24]:
# 정규식 처리
import pandas as pd
from tqdm import tqdm

tqdm.pandas(desc="광고 필터링 진행중")
print("--- '강한 신호' 기반 광고 필터링 시작 ---")

# 전처리 된 df로 처리
original_count = len(df)
print(f"필터링 시작 전 데이터 개수: {original_count}")

# =========================
# 광고 패턴 및 유형 정의
# =========================
STRONG_PATTERNS = [
    # (A) 협찬/광고 고지/해시태그
    r"(원고료\s*를\s*받[아았]|협찬\s*받[아았]|광고\s*표기|#?\s?(광고|협찬)|\bAD\b|\bPR\b)",
    # (B) 직접 행동 유도 CTA / 구매/예약
    r"(바로가기|자세히\s*보기|상세\s*보기|링크\s*클릭|구매\s*하기|신청\s*하기|예약\s*하기|구독\s*하기)",
    # (C) 확실한 광고/유입 목적 URL만 필터링
    r"(smartstore\.naver\.com|linktr\.ee|forms\.gle|bit\.ly|me2\.do|t\.co|shorturl\.at|url\.kr|tinyurl\.com|bitly\.com|naver\.me)",
    # (D) 커머스 핵심 행위/혜택
    r"(장바구니|결제\s*하기|무이자|할부|무료\s*배송|교환|환불)",
    # (E) 할인/특가/가격(명시적 판매)
    r"(\b\d{1,3}\s?%(\s*할인)?|할인\s*\d{1,2}\s*%|할인\s*행사|특가\s*행사)",
    r"((판매가|할인가|행사가|정가)\s*[\d,]+(?:원|원\s*입니다)|\b[\d,]+원\s*(에\s*판매|구매))",
    # (F) 연락처/대표번호
    r"([0-9]{2,3}-[0-9]{3,4}-[0-9]{4})",
    r"\b01[016789][\s\.-]?\d{3,4}[\s\.-]?\d{4}\b",
    r"(\+82[-\s]?(?:10|1[6-9]|2|[3-6][1-5])[-\s]?\d{3,4}[-\s]?\d{4})",
    r"(대표번호|대표전화|고객센터|ARS)\s*\d{2,4}[-]?\d{3,4}[-]?\d{3,4}",
    # (G) 메신저/채널 유도
    r"(open\.kakao\.com|카카오톡\s*채널|카카오\s*채널|오픈채팅|카톡\s*오픈|텔레그램|Telegram|라인\s*ID|카카오\s*ID|카톡\s*ID|네이버\s*톡톡|톡톡|플러스친구|플친)",
    # (H) 부동산 실판매 문의/예약
    r"(분양\s*문의|청약\s*문의|상담전화|상담\s*예약|모델하우스\s*방문\s*예약|견본주택\s*방문)",
    # (I) 희소성/마감 압박
    r"(선착순|마감\s*(임박|주의)|한정\s*(수량|판매)|오늘\s*마감|지금\s*신청)",
    # (J) 교육 영업
    r"(수강생\s*모집|원서\s*접수|합격\s*보장|설명회\s*신청)",
    # (K) 플랫폼/라이브커머스/선물하기
    r"(쿠팡|coupang|스마트스토어|스토어찜|오늘의딜|쇼핑라이브|라이브\s*커머스|라방|선물하기|카카오\s*선물|톡딜|네이버페이|카카오페이|토스|페이코)",
    # (L) B2B 영업/계좌 안내
    r"(단체\s*주문|대량\s*주문|견적\s*요청|납품|B2B|총판|대리점|입점문의|제휴\s*문의)",
    r"(계좌|입금|입금자명|송금|무통장\s*입금)\s*[^\n]{0,12}\d{2,4}[- ]?\d{3,4}[- ]?\d{3,4}",
    # (M) 사전예약/런칭
    r"(오픈\s*기념|얼리버드|사전\s*(구매|등록|신청))",
    # (N) SNS 참여형 이벤트
    r"(댓글\s*이벤트|팔로우\s*이벤트|공유\s*이벤트|리그램|추첨|당첨|경품)",
]

strong_rgx = re.compile("|".join(STRONG_PATTERNS), re.IGNORECASE)

# 1) 광고 필터
is_ad_mask = df['content'].astype(str).progress_apply(lambda x: bool(strong_rgx.search(x)))
df_ad_cleaned   = df[~is_ad_mask].copy()  # 광고가 아닌 데이터
df_ads_removed  = df[ is_ad_mask].copy()  # 광고로 제거된 데이터

print(f"광고 제거 후 남은 데이터: {len(df_ad_cleaned)}")
print(f"광고로 제거된 데이터: {len(df_ads_removed)}")

--- '강한 신호' 기반 광고 필터링 시작 ---
필터링 시작 전 데이터 개수: 211261


광고 필터링 진행중: 100%|██████████| 211261/211261 [02:47<00:00, 1262.84it/s]


광고 제거 후 남은 데이터: 146598
광고로 제거된 데이터: 64663


# 3단계: 결과 저장

In [25]:
# --- 결과 저장 (csv) ---

output_dir = "../data/processed"

out_cleaned_csv = os.path.join(output_dir, f"소셜_데이터_전처리완료.csv")
out_removed_csv = os.path.join(output_dir, f"제거된_광고_데이터.csv")

# CSV 저장
df_ad_cleaned.to_csv(out_cleaned_csv, index=False, encoding="utf-8-sig")
df_ads_removed.to_csv(out_removed_csv, index=False, encoding="utf-8-sig")

print("\n저장 완료!")

print(f"- 광고 제거 데이터(CSV): '{out_cleaned_csv}'")
print(f"- 제거된 광고 데이터(CSV): '{out_removed_csv}'")


저장 완료!
- 광고 제거 데이터(CSV): '../data/processed\소셜_데이터_전처리완료.csv'
- 제거된 광고 데이터(CSV): '../data/processed\제거된_광고_데이터.csv'
